In [135]:
import pandas as pd

In [136]:
fish_df = pd.read_csv('Name_Corrected_All_Seins.csv')

In [137]:
fish_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 4 columns):
Unnamed: 0       892 non-null int64
Date             892 non-null object
Number Caught    727 non-null object
Species          892 non-null object
dtypes: int64(1), object(3)
memory usage: 27.9+ KB


In [138]:
fish_df = fish_df.fillna(1)

In [139]:
fish_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 4 columns):
Unnamed: 0       892 non-null int64
Date             892 non-null object
Number Caught    892 non-null object
Species          892 non-null object
dtypes: int64(1), object(3)
memory usage: 27.9+ KB


In [140]:
fish_df.head()

,Unnamed: 0,Date,Number Caught,Species
0,0,2008-09-13 00:00:00,1,tunicates
1,1,2008-09-13 00:00:00,1,shrimp
2,2,2008-09-13 00:00:00,1,shrimp
3,3,2008-09-13 00:00:00,1,shrimp
4,4,2008-09-13 00:00:00,1,hermit crab


In [141]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [142]:
num_caught_list = []
for num in list(fish_df['Number Caught']):
    if type(num) ==  int:
        num_caught_list.append(num)
    elif is_number(num):
        num_caught_list.append(int(float(num)))
    else:
        number = ''
        started = False
        for char in num:
            if char.isdigit():
                number += char
                started = True
            else:
                if char == '~':
                    print started
                if started:
                    break
        if len(number):
            num_caught_list.append(int(number))
        else:
            if num in ['Numerous', 'TMTC', 'Too Many To Count']:
                num_caught_list.append(30)
            else:
                num_caught_list.append(1)

False
False
False


In [143]:
len(num_caught_list)

892

In [144]:
fish_df['Number Caught'] = num_caught_list

In [145]:
fish_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 4 columns):
Unnamed: 0       892 non-null int64
Date             892 non-null object
Number Caught    892 non-null int64
Species          892 non-null object
dtypes: int64(2), object(2)
memory usage: 27.9+ KB


In [146]:
fish_df.head()

,Unnamed: 0,Date,Number Caught,Species
0,0,2008-09-13 00:00:00,1,tunicates
1,1,2008-09-13 00:00:00,1,shrimp
2,2,2008-09-13 00:00:00,1,shrimp
3,3,2008-09-13 00:00:00,1,shrimp
4,4,2008-09-13 00:00:00,1,hermit crab


In [147]:
fish_names = set(list(fish_df['Species']))

In [148]:
fish_df['Date'] = pd.to_datetime(fish_df['Date'])

In [149]:
catch_by_year = fish_df[['Date', 'Species', 'Number Caught']].groupby(fish_df['Date'].map(lambda x: x.year))

In [150]:
year_fish_count = {}
cat =catch_by_year.get_group(2010).groupby('Species').sum()
cat.index

for group in catch_by_year:
    year_fish_count[group[0]] = []
    fish_caught = catch_by_year.get_group(group[0]).groupby('Species').sum()
    for name in sorted(list(fish_names)):
        if name in list(fish_caught.index):
            year_fish_count[group[0]].append(int(fish_caught.loc[name]))
        else:
            year_fish_count[group[0]].append(0)

In [151]:
#year_fish_count

In [152]:
fish_check_list = pd.DataFrame(data = sorted(list(fish_names)), columns = ['Species'] )

In [153]:
#fish_check_list

In [154]:
years = range(2008,2018)

In [155]:
for year in years:
    fish_check_list[year] =  year_fish_count[year]

In [156]:
fish_check_list
fish_check_list['Total Caught'] = fish_check_list[[2008, 2009, 2010, 2011,2012,2013,2014,2015,2016,2017]].sum(axis=1)

In [157]:
total_by_year_list = ['Total Caught Per Year']

for year in fish_check_list.columns.values:
    if type(year) is int:
        total_by_year_list.append(sum(list(fish_check_list[year])))
total_by_year_list.append(sum(list(fish_check_list['Total Caught'])))
print total_by_year_list
fish_check_list.loc[fish_check_list.shape[0]] = total_by_year_list


['Total Caught Per Year', 17, 16, 123, 1345, 664, 685, 175, 792, 1560, 914, 6291]


In [158]:
#fish_check_list[['Species', 'Total Caught']].sort_values('Total Caught', ascending = False)

In [159]:
fish_check_list['Total Seasons'] =  (fish_check_list[[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017]] > 0).sum(1)
fish_check_list['' ] = ''

In [160]:
fish_check_list = pd.concat([fish_check_list,fish_check_list[['Species', 'Total Caught']].sort_values('Total Caught', ascending = False).reset_index()], axis = 1)

In [161]:
fish_check_list  = fish_check_list.drop('index', axis=1)


In [162]:
fish_check_list.to_csv('Species_Count_By_Year.csv', index = False)